In [2]:
#!pip install langchain-chroma
#!pip install langchain
#!pip install langchain_community
#!pip install langchainhub

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 KB 14.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 KB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.3 MB/s eta 0:00:0000:0100:01
  Using cached onnxruntime-1.18.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)
  Using cached grpcio-1.64.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 KB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 KB 

In [35]:
#!pip install langchain-chroma
#!pip install sentence-transformers

In [25]:
from langchain_community.document_loaders import WebBaseLoader

In [26]:
loader = WebBaseLoader("https://www.foxsports.com/nba/jamal-murray-player-stats")
data = loader.load()
print(data)

[Document(page_content="\n\n\nJamal Murray Stats - NBA | FOX Sports\n\n\n    my favs \n          Access and manage your favorites here\n          \n            DISMISS\n                  Scores Watch UEFA Euro 2024 COPA AMÉRICA Odds Super 6 Stories  \n                  Search\n                     \n                  Sign In\n                    \n                  Account\n                    \n              SPORTS & TEAMS\n             \n              PLAYERS\n             \n              SHOWS\n             \n              PERSONALITIES\n                    SPORTS   \n              SPORTS & TEAMS\n            \n              PLAYERS\n            \n              SHOWS\n            \n              PERSONALITIES\n             \n          SPORTS\n          \n                NFL\n                \n                NCAA FB\n                \n                MLB\n                \n                NBA\n                \n                NCAA BK\n                \n                NASCAR\n     

In [36]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [38]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [39]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/home/luca/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/luca/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
Chroma.from_documents(documents=splits, embedding=embedding_function, persist_directory="./chroma_db")

In [41]:
# PipeLine
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [42]:
from langchain_community.llms import Ollama

In [43]:
model_name = "phi3"
llm = Ollama(model=model_name)
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

/home/luca/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [44]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [45]:
question = "Resume the stats for me"
result = qa_chain({"query": question })

/home/luca/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [48]:
question = "Which stats should he enhance to get a better average points per game?"
result = qa_chain({"query": question })

In [49]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

(' To get a better average points per game, Jamar Murray should focus on '
 'enhancing his scoring efficiency by improving shooting percentages and '
 'increasing the total number of points scored.')


In [47]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(result["result"])

(' Jamal Murray, a player for the Denver Nuggets in the NBA, averages 21.2 '
 'points per game (PPG), which ranks him 31st in the league. He also records '
 'an average of 4.1 rebounds per game (RPG) and 6.5 assists per game (APG), '
 'ranking 15th in the league for assists. Additionally, his field goal '
 'percentage is 55.4%, while he contributes on defense with 0.7 blocks per '
 'game (BPG) and 1 steal per game (SPG). Murray also leads in miscellaneous '
 'stats with a plus-minus of +311.\n'
 '**Response: Jamal Murray averages 21.2 PPG, ranks 31st for points, has 4.1 '
 'RPG and is the 15th leader for APG at 6.5 assists per game, with a field '
 'goal percentage of 55.4%, blocks average of 0.7 per game, steals average of '
 '1 per game, and a miscellaneous stat of +311 in plus-minus ranking.**\n'
 '\n'
 "[Note: The user's requirement for concise answers has been met with a single "
 "sentence summarizing Jamal Murray's statistics.]")
